In [2]:
'https://github.com/GoncaloJardim/ironhack-v4-data-lessons/blob/main/02_data_wrangling_and_retrieval/quest/data-thieves.md'

'https://github.com/GoncaloJardim/ironhack-v4-data-lessons/blob/main/02_data_wrangling_and_retrieval/quest/data-thieves.md'

**Deliverables**

You are required to turn in the following:

    GitHub Repository with scripts and notebooks used, README and a requirements.txt file.
    README file should have a description of your project and how does the pipeline work.
    Requirements file should have all the packages used for someone to run your scripts/notebooks. Always consider that when someone uses your code, they'll do it in a new environment, where the only packages that should be installed are the ones your project requires to be ran. Ideally it should also be specified the package version (ex: scipy==1.4.1)
    Links to any external data you used (should also be on the README file) .
    Slides for your presentation.


**README File structure**

The README will be your paper and exists to act as a guide of your project. From describing the idea to having a walk-through on how to use your scripts, the README is the welcome file for any user that encounters your project.

The structure should be:

    Title of the project
    Introduction to your project.
    Explanation of how your pipeline works (API's + web-scraping depending on each case).
    Links to sources of external data you used to enrich your project (if aplicable).
    Conclusions after your analysis/ product/ feature(s) development.
    Further questions.

**Presentation**

You will have 10 minutes to present your project. The below are some ideas for slides you could include in your presentation; those marked with an (M) are mandatory!

    (M) Title of the project
    (M) Business idea.
    (M) Technical developments.
    Main challenges & strengths.
    Product / Feature showcasing.
    Main insights.
    Questions you couldn't answer.
    Something funny that happened during the project.
    Things you learned during this project.
    If you could start from scratch, what would you do differently?


In [2]:
#!pip install python-dotenv
import pandas as pd
import requests


In [3]:
maps = requests.get('https://www.google.com/maps/search/?api=1&parameters')
print("map:", maps.status_code)

map: 200


In [5]:
info = maps.json()
info

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv()

my_key = os.getenv("API_KEY")
